In [3]:
import requests
import bs4
import json

In [4]:
# start with the first page and get all the links (of posts) that we need
#url = 'https://csn.cancer.org/forum/127' #breast cancer
#url = 'https://csn.cancer.org/forum/165'  #brain cancer
url = 'https://csn.cancer.org/forum/165' #breast cancer
resp = requests.get(url)
soup = bs4.BeautifulSoup(resp.text)
links = []
for tag in soup.find_all('a'):
    link = tag.get('href')
    try:
        if link.startswith('/node'):
            links.append(link)
    except:
        pass
print links

['/node/155606', '/node/309176', '/node/233406', '/node/309097', '/node/215156', '/node/308994', '/node/186066', '/node/231624', '/node/162906', '/node/308397', '/node/252316', '/node/172955', '/node/308842', '/node/308574', '/node/308836', '/node/308208', '/node/144004', '/node/305590', '/node/308634', '/node/188739', '/node/308466', '/node/187973', '/node/308174', '/node/241772', '/node/307670', '/node/308158', '/node/307286', '/node/169168', '/node/308059', '/node/306679', '/node/245386', '/node/157353', '/node/220186', '/node/307881', '/node/307811', '/node/306335', '/node/307705', '/node/307683', '/node/307455', '/node/307411', '/node/304634', '/node/226484', '/node/307162', '/node/170054', '/node/306862', '/node/306688', '/node/302141', '/node/306599', '/node/306597', '/node/157642']


C:\Anaconda2\envs\gl-env\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
#define functions to extract all needed variables
def username():
    users = []
    i = 1
    while i < 100:
        try:
            user = str(soup.find_all("span", { "class" : "username" })).split('<span class="username">')[i].split('</span>')[0]
            #print user
            users.append(user)
        except:
            pass
        i += 1
    return users

def posttime():
    post_times = []
    post_time = str(soup.find_all("div", { "class" : "submitted" })).split('>')[1].split('-')[0]
    post_times.append(post_time)

    i = 1
    while i < 100:
        try:
            reply_time = str(soup.find_all("span", { "class" : "submitted" })).split('<span class="submitted">')[i].split('-')[0]
            post_times.append(reply_time)
        except:
            pass
        i += 1
    return post_times

def text():
    texts = []
    i = 0
    while i < 100:
        try:
            text = str(soup.find_all("div", { "class" : "field-item even" })).split('</p')[i].split('<p>')[1]
            #print text
            texts.append(text)
        except:
            pass
        i += 1
    return texts

def postcount():
    post_counts = []
    i = 1
    while i < 100:
        try:
            s = str(soup.find_all("span", { "class" : "author-posts" })).split('Posts: ')[i]
            post_count = int(filter(str.isdigit, s))
            #print text
            post_counts.append(post_count)
        except:
            pass
        i += 1
    return post_counts

def jointime():
    join_times = []
    i = 1
    while i < 100:
        try:
            s1 = str(soup.find_all("span", { "class" : "author-regdate" })).split('Joined: ')[i]
            s2 = str(filter(str.isdigit, s1))
            join_time = s1.split()[0]+' '+s2
            join_times.append(join_time)
        except:
            pass
        i += 1
    return join_times

def reply():
    replies = []
    i = 1
    while i < 100:
        try:
            reply = {'user':users[i], 'post_time':post_times[i], 'post_count':post_counts[i], 'text':texts[i], 'join_time':join_times[i]}
            replies.append(reply)
        except:
            pass
        i += 1
    return replies

In [4]:
#work on each of the links that we got
posts = {}
for link in links:
    url = 'https://csn.cancer.org'+link
    resp = requests.get(url) 
    soup = bs4.BeautifulSoup(resp.text)
    
    #now extract all needed variables
    users = username()
    post_times = posttime()
    post_counts = postcount()
    texts = text()
    join_times = jointime()
    replies = reply()
    
    try:
        posts[link] = {'user':users[0], 
                                 'post_time':post_times[0], 
                                 'post_count':post_counts[0],
                                 'text':texts[0],
                                 'join_time':join_times[0],
                                 'reply':replies}
    except:
        pass

KeyboardInterrupt: 

In [ ]:
print len(posts)
print posts['/node/221036']['user']
print posts['/node/221036']['reply'][1]['post_time']

In [51]:
# save it to json data
with open('data.txt', 'w') as outfile:
    json.dump(posts, outfile)

In [2]:
# load json file
with open('data.txt') as json_file:  
    data = json.load(json_file)
print data['/node/221036']['user']
print data['/node/307678']['reply'][1]['post_time']

beachlovernh
Feb 02, 2017 


In [17]:
# now work on the remaining pages and extract all links and info
#posts = {}
for i in range(10,20):
    print i
    url =  'https://csn.cancer.org/forum/127?page='+str(i)
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.text)
    links = []
    for tag in soup.find_all('a'):
        link = tag.get('href')
        try:
            if link.startswith('/node'):
                links.append(link)
        except:
            pass

    for link in links:
        url = 'https://csn.cancer.org'+link
        print url
        resp = requests.get(url) 
        soup = bs4.BeautifulSoup(resp.text)

        #now extract all needed variables
        users = username()
        post_times = posttime()
        post_counts = postcount()
        texts = text()
        join_times = jointime()
        replies = reply()
        
        try:       
            posts[link] = {'user':users[0], 
                                     'post_time':post_times[0], 
                                     'post_count':post_counts[0],
                                     'text':texts[0],
                                     'join_time':join_times[0],
                                     'reply':replies}
        except:
            pass

10
https://csn.cancer.org/node/299767
https://csn.cancer.org/node/299980
https://csn.cancer.org/node/299994
https://csn.cancer.org/node/299986
https://csn.cancer.org/node/299969
https://csn.cancer.org/node/299480
https://csn.cancer.org/node/299065
https://csn.cancer.org/node/299786
https://csn.cancer.org/node/298529
https://csn.cancer.org/node/299836
https://csn.cancer.org/node/299878
https://csn.cancer.org/node/299600
https://csn.cancer.org/node/299820
https://csn.cancer.org/node/299542
https://csn.cancer.org/node/293660
https://csn.cancer.org/node/299812
https://csn.cancer.org/node/299654
https://csn.cancer.org/node/299772
https://csn.cancer.org/node/299556
https://csn.cancer.org/node/210796
https://csn.cancer.org/node/299678
https://csn.cancer.org/node/299594
https://csn.cancer.org/node/228399
https://csn.cancer.org/node/299718
https://csn.cancer.org/node/299680
https://csn.cancer.org/node/299615
https://csn.cancer.org/node/298796
https://csn.cancer.org/node/299617
https://csn.cance

In [18]:
with open('data3.txt', 'w') as outfile:
    json.dump(posts, outfile)

In [ ]:
import requests
import bs4
import json

In [9]:
#define functions to extract all needed variables
def username():
    users = []
    i = 1
    while i < 100:
        try:
            user = str(soup.find_all("span", { "class" : "username" })).split('<span class="username">')[i].split('</span>')[0]
            #print user
            users.append(user)
        except:
            pass
        i += 1
    return users

def posttime():
    post_times = []
    post_time = str(soup.find_all("div", { "class" : "submitted" })).split('>')[1].split('-')[0]
    post_times.append(post_time)

    i = 1
    while i < 100:
        try:
            reply_time = str(soup.find_all("span", { "class" : "submitted" })).split('<span class="submitted">')[i].split('-')[0]
            post_times.append(reply_time)
        except:
            pass
        i += 1
    return post_times

def text():
    texts = []
    i = 0
    while i < 100:
        try:
            text = str(soup.find_all("div", { "class" : "field-item even" })).split('<div class="field-item even"></p>')[i].split('<p></div>')[1]
            #print text
            texts.append(text)
        except:
            pass
        i += 1
    return texts

def postcount():
    post_counts = []
    i = 1
    while i < 100:
        try:
            s = str(soup.find_all("span", { "class" : "author-posts" })).split('Posts: ')[i]
            post_count = int(filter(str.isdigit, s))
            #print text
            post_counts.append(post_count)
        except:
            pass
        i += 1
    return post_counts

def jointime():
    join_times = []
    i = 1
    while i < 100:
        try:
            s1 = str(soup.find_all("span", { "class" : "author-regdate" })).split('Joined: ')[i]
            s2 = str(filter(str.isdigit, s1))
            join_time = s1.split()[0]+' '+s2
            join_times.append(join_time)
        except:
            pass
        i += 1
    return join_times

def reply():
    replies = []
    i = 1
    while i < 100:
        try:
            reply = {'user':users[i], 'post_time':post_times[i], 'post_count':post_counts[i], 'text':texts[i], 'join_time':join_times[i]}
            replies.append(reply)
        except:
            pass
        i += 1
    return replies

In [31]:
#emotional support

for i in range(8,0,-1):
    posts = {}
    print i
    url =  'https://csn.cancer.org/forum/137?page='+str(i)
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.text)
    links = []
    for tag in soup.find_all('a'):
        link = tag.get('href')
        try:
            if link.startswith('/node'):
                links.append(link)
        except:
            pass

    for link in links:
        url = 'https://csn.cancer.org'+link
        print url
        resp = requests.get(url) 
        soup = bs4.BeautifulSoup(resp.text)

        #now extract all needed variables
        try:
            users = username()
            post_times = posttime()
            post_counts = postcount()
            texts = text()
            join_times = jointime()
            replies = reply()
        except:
            pass  
        
        try:       
            posts[link] = {'user':users[0], 
                                     'post_time':post_times[0], 
                                     'post_count':post_counts[0],
                                     'text':texts[0],
                                     'join_time':join_times[0],
                                     'reply':replies}
        except:
            pass
    with open('em_data'+str(i)+'.txt', 'w') as outfile:
        json.dump(posts, outfile)

8
https://csn.cancer.org/node/220312
https://csn.cancer.org/node/220310
https://csn.cancer.org/node/220169
https://csn.cancer.org/node/219661
https://csn.cancer.org/node/219051
https://csn.cancer.org/node/219456
https://csn.cancer.org/node/219238
https://csn.cancer.org/node/219341
https://csn.cancer.org/node/219089
https://csn.cancer.org/node/216734
https://csn.cancer.org/node/212505
https://csn.cancer.org/node/218502
https://csn.cancer.org/node/218557
https://csn.cancer.org/node/217192
https://csn.cancer.org/node/218068
https://csn.cancer.org/node/218258
https://csn.cancer.org/node/213230
https://csn.cancer.org/node/217026
https://csn.cancer.org/node/216482
https://csn.cancer.org/node/217265
https://csn.cancer.org/node/216566
https://csn.cancer.org/node/212032
https://csn.cancer.org/node/215404
https://csn.cancer.org/node/215418
https://csn.cancer.org/node/215728
https://csn.cancer.org/node/215563
https://csn.cancer.org/node/215564
https://csn.cancer.org/node/215308
https://csn.cancer

In [32]:
with open('em_data4.txt', 'w') as outfile:
    json.dump(posts, outfile)

# Start from here

In [1]:
def username():
    users = []
    i = 1
    while i < 70:
        try:
            user = str(soup.find_all("span", { "class" : "username" })).split('<span class="username">')[i].split('</span>')[0]
            #print user
            users.append(user)
        except:
            pass
        i += 1
    return users

def posttime():
    post_times = []
    post_time = str(soup.find_all("div", { "class" : "submitted" })).split('>')[1].split('-')[0]
    post_times.append(post_time)

    i = 1
    while i < 70:
        try:
            reply_time = str(soup.find_all("span", { "class" : "submitted" })).split('<span class="submitted">')[i].split('-')[0]
            post_times.append(reply_time)
        except:
            pass
        i += 1
    return post_times

def text():
    texts = []
    i = 1
    while i < 70:
        try:
            text = str(soup.find_all("div", { "class" : "field-item even" })).split('<div class="field-item even"><p>')[i].split('</p></div>')[0]
            #print text
            texts.append(text)
        except:
            pass
        i += 1
    return texts

def postcount():
    post_counts = []
    i = 1
    while i < 70:
        try:
            s = str(soup.find_all("span", { "class" : "author-posts" })).split('Posts: ')[i]
            post_count = int(filter(str.isdigit, s))
            #print text
            post_counts.append(post_count)
        except:
            pass
        i += 1
    return post_counts

def jointime():
    join_times = []
    i = 1
    while i < 70:
        try:
            s1 = str(soup.find_all("span", { "class" : "author-regdate" })).split('Joined: ')[i]
            s2 = str(filter(str.isdigit, s1))
            join_time = s1.split()[0]+' '+s2
            join_times.append(join_time)
        except:
            pass
        i += 1
    return join_times

def reply():
    replies = []
    i = 1
    while i < 70:
        try:
            reply = {'user':users[i], 'post_time':post_times[i], 'text':texts[i]}
            replies.append(reply)
        except:
            pass
        i += 1
    return replies

In [10]:
"""""
posts = {}
#url = 'https://csn.cancer.org/forum/165'
url = 'https://csn.cancer.org/forum/165?page=1'
resp = requests.get(url)
soup = bs4.BeautifulSoup(resp.text)
links = []
for tag in soup.find_all('a'):
    link = tag.get('href')
    try:
        if link.startswith('/node'):
            links.append(link)
    except:
        pass

for link in links:
    url = 'https://csn.cancer.org'+link
    print url
    resp = requests.get(url) 
    soup = bs4.BeautifulSoup(resp.text)

    #now extract all needed variables
    users = username()
    post_times = posttime()
    texts = text()
    replies = reply()

    try:       
        posts[link] = {'user':users[0], 
                                 'post_time':post_times[0], 
                                 'text':texts[0],
                                 'reply':replies}
    except:
        pass
    
    j = 1
    while j < 7:
        url = 'https://csn.cancer.org'+link+'?page='+str(j)
        resp = requests.get(url) 
        soup = bs4.BeautifulSoup(resp.text)
        try:
            users = username()
            post_times = posttime()
            texts = text()
            replies = reply()
            
            posts[link]['reply'].append(replies)
        except:
            pass
        j += 1
            
with open('D:/breast/brain_data.txt', 'w') as outfile:
    json.dump(posts, outfile)        

https://csn.cancer.org/node/306557
https://csn.cancer.org/node/247204
https://csn.cancer.org/node/306157
https://csn.cancer.org/node/304189
https://csn.cancer.org/node/297691
https://csn.cancer.org/node/136188
https://csn.cancer.org/node/150775
https://csn.cancer.org/node/211067
https://csn.cancer.org/node/296736
https://csn.cancer.org/node/305512
https://csn.cancer.org/node/299252
https://csn.cancer.org/node/139723
https://csn.cancer.org/node/305230
https://csn.cancer.org/node/305271
https://csn.cancer.org/node/169907
https://csn.cancer.org/node/303932
https://csn.cancer.org/node/304891
https://csn.cancer.org/node/304771
https://csn.cancer.org/node/304269
https://csn.cancer.org/node/217488
https://csn.cancer.org/node/300276
https://csn.cancer.org/node/303172
https://csn.cancer.org/node/178925
https://csn.cancer.org/node/303597
https://csn.cancer.org/node/303740
https://csn.cancer.org/node/292164
https://csn.cancer.org/node/303258
https://csn.cancer.org/node/303357
https://csn.cancer.o

In [32]:
#posts['/node/141428']['reply']

In [29]:
posts = {}
url = 'https://csn.cancer.org/forum/165'
#url = 'https://csn.cancer.org/forum/165?page=5'
resp = requests.get(url)
soup = bs4.BeautifulSoup(resp.text)
links = []
for tag in soup.find_all('a'):
    link = tag.get('href')
    try:
        if link.startswith('/node'):
            links.append(link)
    except:
        pass

for link in links:
    url = 'https://csn.cancer.org'+link
    print url
    resp = requests.get(url) 
    soup = bs4.BeautifulSoup(resp.text)

    #now extract all needed variables
    users = username()
    post_times = posttime()
    texts = text()
    replies = reply()

    try:       
        posts[link] = {'user':users[0], 
                                 'post_time':post_times[0], 
                                 'text':texts[0],
                                 'reply':replies}
    except:
        pass
    
    if link == "/node/157642" or link == '/node/233406':
        for j in range(1, 2):
            url = 'https://csn.cancer.org'+link+'?page='+str(j)
            resp = requests.get(url) 
            soup = bs4.BeautifulSoup(resp.text)
            try:
                users = username()
                post_times = posttime()
                texts = text()
                replies = reply()

                posts[link]['reply'].extend(replies)
            except:
                pass
            
    elif link == "/node/220186":
        for j in range(1, 3):
            url = 'https://csn.cancer.org'+link+'?page='+str(j)
            resp = requests.get(url) 
            soup = bs4.BeautifulSoup(resp.text)
            try:
                users = username()
                post_times = posttime()
                texts = text()
                replies = reply()

                posts[link]['reply'].extend(replies)
            except:
                pass

    elif link == "/node/169168" or link == '/node/186066' or link == '/node/155606':
        for j in range(1, 4):   #1,2,3,
            url = 'https://csn.cancer.org'+link+'?page='+str(j)
            resp = requests.get(url) 
            soup = bs4.BeautifulSoup(resp.text)
            try:
                users = username()
                post_times = posttime()
                texts = text()
                replies = reply()

                posts[link]['reply'].extend(replies)
            except:
                pass
            
    elif link == "/node/157353":
        for j in range(1, 5):
            url = 'https://csn.cancer.org'+link+'?page='+str(j)
            resp = requests.get(url) 
            soup = bs4.BeautifulSoup(resp.text)
            try:
                users = username()
                post_times = posttime()
                texts = text()
                replies = reply()

                posts[link]['reply'].extend(replies)
            except:
                pass
            
    elif link == "/node/162906":
        for j in range(1, 6):
            url = 'https://csn.cancer.org'+link+'?page='+str(j)
            resp = requests.get(url) 
            soup = bs4.BeautifulSoup(resp.text)
            try:
                users = username()
                post_times = posttime()
                texts = text()
                replies = reply()

                posts[link]['reply'].extend(replies)
            except:
                pass
            
with open('D:/breast/brain_data0.txt', 'w') as outfile:
    json.dump(posts, outfile)     

https://csn.cancer.org/node/155606
https://csn.cancer.org/node/309176
https://csn.cancer.org/node/233406
https://csn.cancer.org/node/309097
https://csn.cancer.org/node/215156
https://csn.cancer.org/node/308994
https://csn.cancer.org/node/186066
https://csn.cancer.org/node/231624
https://csn.cancer.org/node/162906
https://csn.cancer.org/node/308397
https://csn.cancer.org/node/252316
https://csn.cancer.org/node/172955
https://csn.cancer.org/node/308842
https://csn.cancer.org/node/308574
https://csn.cancer.org/node/308836
https://csn.cancer.org/node/308208
https://csn.cancer.org/node/144004
https://csn.cancer.org/node/305590
https://csn.cancer.org/node/308634
https://csn.cancer.org/node/188739
https://csn.cancer.org/node/308466
https://csn.cancer.org/node/187973
https://csn.cancer.org/node/308174
https://csn.cancer.org/node/241772
https://csn.cancer.org/node/307670
https://csn.cancer.org/node/308158
https://csn.cancer.org/node/307286
https://csn.cancer.org/node/169168
https://csn.cancer.o

In [31]:
#posts['/node/155606']['reply']

In [28]:
for i in range(1,2):
    print i

1
